<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

*  [Bag Documenation](http://dask.pydata.org/en/latest/bag.html)
*  [Bag API](http://dask.pydata.org/en/latest/bag-api.html)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [1]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [2]:
# each element is a text file, where each line is a JSON object
# note that the compression is handles automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Xavier", "transactions": [{"transaction-id": 10963, "amount": 9559}, {"transaction-id": 12412, "amount": 9104}, {"transaction-id": 14088, "amount": 9265}, {"transaction-id": 14320, "amount": 10200}, {"transaction-id": 15154, "amount": 9833}, {"transaction-id": 19987, "amount": 9615}, {"transaction-id": 20285, "amount": 9945}, {"transaction-id": 22727, "amount": 9276}, {"transaction-id": 29679, "amount": 9386}, {"transaction-id": 32263, "amount": 9215}, {"transaction-id": 40963, "amount": 9631}, {"transaction-id": 45759, "amount": 9618}, {"transaction-id": 53930, "amount": 9140}, {"transaction-id": 54775, "amount": 9205}, {"transaction-id": 58636, "amount": 9374}, {"transaction-id": 60618, "amount": 9477}, {"transaction-id": 61700, "amount": 9568}, {"transaction-id": 62516, "amount": 9397}, {"transaction-id": 64376, "amount": 9321}, {"transaction-id": 66431, "amount": 9187}, {"transaction-id": 70596, "amount": 9453}, {"transaction-id": 73026, "amount": 8913}, {"tran

In [3]:
# # Requires `s3fs` library
# # each partition is a remote CSV text file
# b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv')
# b.take(1)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [4]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda-..., npartitions=10>

In [5]:
%%time
# blocking form: wait for completion (which is very fast in this case)
c.compute()

Wall time: 434 ms


[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [6]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Xavier", "transactions": [{"transaction-id": 10963, "amount": 9559}, {"transaction-id": 12412, "amount": 9104}, {"transaction-id": 14088, "amount": 9265}, {"transaction-id": 14320, "amount": 10200}, {"transaction-id": 15154, "amount": 9833}, {"transaction-id": 19987, "amount": 9615}, {"transaction-id": 20285, "amount": 9945}, {"transaction-id": 22727, "amount": 9276}, {"transaction-id": 29679, "amount": 9386}, {"transaction-id": 32263, "amount": 9215}, {"transaction-id": 40963, "amount": 9631}, {"transaction-id": 45759, "amount": 9618}, {"transaction-id": 53930, "amount": 9140}, {"transaction-id": 54775, "amount": 9205}, {"transaction-id": 58636, "amount": 9374}, {"transaction-id": 60618, "amount": 9477}, {"transaction-id": 61700, "amount": 9568}, {"transaction-id": 62516, "amount": 9397}, {"transaction-id": 64376, "amount": 9321}, {"transaction-id": 66431, "amount": 9187}, {"transaction-id": 70596, "amount": 9453}, {"transaction-id": 73026, "amount": 8913}, {"tran

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [7]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Xavier',
  'transactions': [{'transaction-id': 10963, 'amount': 9559},
   {'transaction-id': 12412, 'amount': 9104},
   {'transaction-id': 14088, 'amount': 9265},
   {'transaction-id': 14320, 'amount': 10200},
   {'transaction-id': 15154, 'amount': 9833},
   {'transaction-id': 19987, 'amount': 9615},
   {'transaction-id': 20285, 'amount': 9945},
   {'transaction-id': 22727, 'amount': 9276},
   {'transaction-id': 29679, 'amount': 9386},
   {'transaction-id': 32263, 'amount': 9215},
   {'transaction-id': 40963, 'amount': 9631},
   {'transaction-id': 45759, 'amount': 9618},
   {'transaction-id': 53930, 'amount': 9140},
   {'transaction-id': 54775, 'amount': 9205},
   {'transaction-id': 58636, 'amount': 9374},
   {'transaction-id': 60618, 'amount': 9477},
   {'transaction-id': 61700, 'amount': 9568},
   {'transaction-id': 62516, 'amount': 9397},
   {'transaction-id': 64376, 'amount': 9321},
   {'transaction-id': 66431, 'amount': 9187},
   {'transaction-id': 70596, 'am

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [8]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 48,
  'name': 'Alice',
  'transactions': [{'transaction-id': 579, 'amount': 399},
   {'transaction-id': 760, 'amount': 629},
   {'transaction-id': 1581, 'amount': 1060},
   {'transaction-id': 1620, 'amount': 934},
   {'transaction-id': 2854, 'amount': 675},
   {'transaction-id': 3108, 'amount': 662},
   {'transaction-id': 3691, 'amount': 746},
   {'transaction-id': 3859, 'amount': 829},
   {'transaction-id': 5024, 'amount': 788},
   {'transaction-id': 5160, 'amount': 679},
   {'transaction-id': 5920, 'amount': 716},
   {'transaction-id': 6343, 'amount': 874},
   {'transaction-id': 7900, 'amount': 785},
   {'transaction-id': 7930, 'amount': 992},
   {'transaction-id': 9419, 'amount': 885},
   {'transaction-id': 13982, 'amount': 872},
   {'transaction-id': 14783, 'amount': 1012},
   {'transaction-id': 15173, 'amount': 486},
   {'transaction-id': 16483, 'amount': 676},
   {'transaction-id': 19166, 'amount': 481},
   {'transaction-id': 19525, 'amount': 891},
   {'transaction-id': 1

In [10]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))


({'name': 'Alice', 'count': 145},
 {'name': 'Alice', 'count': 33},
 {'name': 'Alice', 'count': 109},
 {'name': 'Alice', 'count': 1},
 {'name': 'Alice', 'count': 38})

In [11]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(145, 33, 109, 1, 38)

In [13]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

95.18239795918367

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [14]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 579, 'amount': 399},
  {'transaction-id': 760, 'amount': 629},
  {'transaction-id': 1581, 'amount': 1060},
  {'transaction-id': 1620, 'amount': 934},
  {'transaction-id': 2854, 'amount': 675},
  {'transaction-id': 3108, 'amount': 662},
  {'transaction-id': 3691, 'amount': 746},
  {'transaction-id': 3859, 'amount': 829},
  {'transaction-id': 5024, 'amount': 788},
  {'transaction-id': 5160, 'amount': 679},
  {'transaction-id': 5920, 'amount': 716},
  {'transaction-id': 6343, 'amount': 874},
  {'transaction-id': 7900, 'amount': 785},
  {'transaction-id': 7930, 'amount': 992},
  {'transaction-id': 9419, 'amount': 885},
  {'transaction-id': 13982, 'amount': 872},
  {'transaction-id': 14783, 'amount': 1012},
  {'transaction-id': 15173, 'amount': 486},
  {'transaction-id': 16483, 'amount': 676},
  {'transaction-id': 19166, 'amount': 481},
  {'transaction-id': 19525, 'amount': 891},
  {'transaction-id': 19539, 'amount': 592},
  {'transaction-id': 19628, 'amount': 906},
  {

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 579, 'amount': 399},
 {'transaction-id': 760, 'amount': 629},
 {'transaction-id': 1581, 'amount': 1060})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(399, 629, 1060)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

580.5393109363065

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [20]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Dan', 'Bob']), (5, ['Alice', 'Edith', 'Frank'])]

In [25]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, sorted(v))).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [26]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [62]:
is_even = lambda x: x % 2

b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [54]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 94), ('Alice', 95), ('Alice', 96), ('Alice', 109), ('Alice', 187), ('Alice', 203), ('Bob', 92), ('Bob', 92), ('Bob', 92), ('Bob', 93), ('Bob', 93), ('Bob', 96), ('Bob', 107), ('Bob', 108), ('Charlie', 132), ('Charlie', 132), ('Charlie', 132), ('Charlie', 154), ('Charlie', 264), ('Charlie', 286), ('Dan', 161), ('Dan', 162), ('Dan', 162), ('Dan', 162), ('Dan', 188), ('Dan', 189), ('Dan', 324), ('Edith', 114), ('Edith', 114), ('Edith', 114), ('Edith', 114), ('Edith', 133), ('Edith', 133), ('Edith', 228), ('Frank', 132), ('Frank', 132), ('Frank', 132), ('Frank', 132), ('Frank', 132), ('Frank', 132), ('Frank', 154), ('Frank', 154), ('George', 126), ('George', 126), ('George', 126), ('George', 126), ('George', 126), ('George', 147), ('George', 273), ('Hannah', 132), ('Hannah', 132), ('Hannah', 132), ('Hannah', 154), ('Hannah', 264), ('Hannah', 286), ('Ingrid', 135), ('Ingrid', 135), ('Ingrid', 136), ('Ingrid', 136), ('Ingrid', 136), ('Ingrid', 138), ('Ingrid', 156), ('Ingrid', 161

In [55]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 784), ('Bob', 773), ('Charlie', 1100), ('Dan', 1348), ('Edith', 950), ('Frank', 1100), ('George', 1050), ('Hannah', 1100), ('Ingrid', 1133), ('Jerry', 900), ('Kevin', 750), ('Laura', 865), ('Michael', 950), ('Norbert', 800), ('Oliver', 1000), ('Patricia', 1100), ('Quinn', 500), ('Ray', 1100), ('Sarah', 550), ('Tim', 1197), ('Ursula', 1299), ('Victor', 750), ('Wendy', 1000), ('Xavier', 1100), ('Yvonne', 927), ('Zelda', 744)]
Wall time: 1.46 s


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [76]:
# Your code here...
sum_dict = lambda _, d: sum(pr["amount"] for pr in d["transactions"])
result = js.foldby(
    key="name",
    binop=sum_dict,
    initial=0,
    combine=add,
    combine_initial=0).compute()
print(sorted(result, key=lambda x: x[1]))

[('Ingrid', -10157561), ('Ursula', -2541352), ('Bob', -1536054), ('Patricia', -781647), ('Ray', -489983), ('Laura', -71740), ('Victor', -8797), ('Alice', 16188), ('George', 23985), ('Jerry', 69533), ('Frank', 121512), ('Wendy', 984465), ('Quinn', 1055516), ('Yvonne', 1403506), ('Hannah', 2011685), ('Oliver', 2156096), ('Charlie', 2475279), ('Tim', 2600699), ('Xavier', 3204069), ('Edith', 6573019), ('Kevin', 7334691), ('Norbert', 7506057), ('Dan', 17832820), ('Zelda', 20168384), ('Sarah', 28288116), ('Michael', 66513784)]


## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [77]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Xavier,"[{'transaction-id': 10963, 'amount': 9559}, {'..."
1,1,Kevin,"[{'transaction-id': 867, 'amount': -173}, {'tr..."
2,2,George,"[{'transaction-id': 778, 'amount': 515}, {'tra..."
3,3,Michael,"[{'transaction-id': 3975, 'amount': -213}, {'t..."
4,4,Frank,"[{'transaction-id': 144, 'amount': 286}, {'tra..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [86]:
%time df1.groupby('name')["id"].count().compute().head()

Wall time: 9.2 s


name
Alice       784
Bob         773
Charlie    1100
Dan        1348
Edith       950
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [87]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Xavier', 'amount': 9559, 'transaction-id': 10963},
 {'id': 0, 'name': 'Xavier', 'amount': 9104, 'transaction-id': 12412},
 {'id': 0, 'name': 'Xavier', 'amount': 9265, 'transaction-id': 14088})

In [88]:
df = transactions.to_dataframe()
df.head()

,amount,id,name,transaction-id
0,9559,0,Xavier,10963
1,9104,0,Xavier,12412
2,9265,0,Xavier,14088
3,10200,0,Xavier,14320
4,9833,0,Xavier,15154


In [89]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

Wall time: 17.2 s


name
Alice        74623
Bob         104679
Charlie     294415
Dan         271175
Edith       137374
Frank       247804
George      234685
Hannah      198911
Ingrid      170481
Jerry       161361
Kevin       152166
Laura       130995
Michael     232849
Norbert     183858
Oliver      144574
Patricia    287110
Quinn       129714
Ray         232809
Sarah       103200
Tim         279477
Ursula      299030
Victor      147911
Wendy       121062
Xavier      247053
Yvonne      271175
Zelda       141509
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

### See also


*  [Blogpost analyzing github data on EC2](https://www.continuum.io/content/dask-distributed-and-anaconda-cluster)